## Advise re identity architecture

Initialize

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha"

#r "nuget: System.Linq.Async, 6.0.1"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;

using Kernel = Microsoft.SemanticKernel.Kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();
#pragma warning disable SKEXP0011, SKEXP0003, SKEXP0052
var memoryBuilder = new MemoryBuilder();
memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(
    "mrtextembeddingada002",
    azureEndpoint, 
    apiKey)
.WithMemoryStore(new VolatileMemoryStore());   
var memory = memoryBuilder.Build();            

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha System.Linq.Async, 6.0.1

Define/load plugin to facilitate use of chat history

In [2]:
using Microsoft.SemanticKernel.Plugins.Memory;

kernel.Plugins.Clear();

#pragma warning disable SKEXP0052

// TextMemoryPlugin provides the "recall" function
kernel.ImportPluginFromObject(new TextMemoryPlugin(memory));
var appFunction = kernel.CreateFunctionFromPrompt(File.ReadAllText("./ApplicationDiscovery.txt"), new OpenAIPromptExecutionSettings { MaxTokens = 200, Temperature = 0.8 });
var userFunction = kernel.CreateFunctionFromPrompt(File.ReadAllText("./UserIdentity.txt"), new OpenAIPromptExecutionSettings { MaxTokens = 200, Temperature = 0.0 });

Define arguments

In [3]:
#pragma warning disable SKEXP0052
var arguments = new KernelArguments();
arguments[TextMemoryPlugin.CollectionParam] = "history";
arguments[TextMemoryPlugin.LimitParam] = "2";
arguments[TextMemoryPlugin.RelevanceParam] = "0.8";

Define chat item

In [4]:
var history = "";
arguments["history"] = history;
Func<string, Task> Chat = async (string input) => {
    arguments["userInput"] = input;
    var answer = await userFunction.InvokeAsync(kernel, arguments);
    var result = $"\nUser: {input}\nassistant: {answer}\n";
    history += result;
    arguments["history"] = history;
    Console.WriteLine(result);
};

Define chat loop

In [5]:
using System.Globalization;
using Microsoft.DotNet.Interactive;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;
do
{
    var user = await InteractiveKernel.GetInputAsync("Please enter your question or response");
    if (String.IsNullOrEmpty(user) || user.StartsWith("q", true, CultureInfo.CurrentCulture)) break;
    await Chat(user);
} while (true);
Console.WriteLine("Finished.");


User: This is a console app for our employees
assistant: Based on the information provided, I would recommend using Azure AD Enterprise, optionally with B2B, for authentication and authorization of users for your console application. This option allows you to have control over who can access the application and manage the authorization process. Additionally, if you need to collaborate with external users, Azure AD B2B can be used to invite them to access the application.


User: This is a web SaaS application
assistant: Based on the information provided, I would recommend using Azure AD Multi-tenant for authentication and authorization of users for your web SaaS application. This option allows each company to control its own authorization and all companies can use their Azure AD directories to manage access to the application.


User: This is a web application
assistant: Based on the information provided, I would recommend using Azure AD B2C for authentication and authorization of use

Some tests

In [13]:
await Chat("Here is my system description: it consists of a single web application showing html content.");


User: Here is my system description: it consists of a single web application showing html content.
ChatBot: Got it. So your system consists of a single web application that shows HTML content. Is that correct?

